In [2]:
# Google news
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

link = 'https://news.google.com/topics/CAAqIggKIhxDQkFTRHdvSkwyMHZNREZqY0hsNUVnSmxiaWdBUAE?hl=en-IN&gl=IN&ceid=IN%3Aen'      

source = requests.get(link).text
soup = BeautifulSoup(source,'lxml')

newsHeader, news, link = [], '', ''


mainDivs = soup.find_all('div',{'class':'DBQmFf NclIid BL5WZb Oc0wGc xP6mwf j7vNaf'})
for i in mainDivs:
    try:
        h = i.find('div',{'class':'cp7Yvc'})
        print('###' + h.text+ ':\n')
        try:
            more = h.find('a')
            print('More Such News: https://news.google.com/'+more['href'][2:])
            print()
        except:
            pass
        div = i.find_all('div',{'class':'xrnccd tcvRoe R7GTQ keNKEd j7vNaf'})
        a = i.find_all('a',{'class':'VDXfz'})
        for j in div:
            headlines = j.find_all('h3')
            refs = j.find_all('a',{'class':'wEwyrc AVN2gc uQIVzc Sksgp'})
            time = j.find_all('time')           
            for k,l,m,n in zip(headlines,refs,time,a):
                news = (k.text+'\nReference: '+l.text+'\nUpdated: '+m.text+'\n',' For more: https://news.google.com/'+n['href'][2:])
                print(news)
                print()
                newsHeader.append(news)
                news = ''
                
        print('-'*50)
        
    except:
        print()

In [1]:
# coding: utf-8

import tweepy
import json
import pandas as pd
from langdetect import detect
from datetime import datetime 
from datetime import timedelta

consumer_key = "cFf7im7BH68xO9qh3zEsv3nFz"
consumer_secret = "2QyodEVK63XYE5D9RFPAo0I53rhBOsNocQGpGB8rapmEqxDnJi"
access_key = "1265319352795975680-EJBIU55ZHZnjn8svR420cqVuU9evRL"
access_secret = "i8h2utF6b2l2Uh7Vpg6c2mnPOO1CuthGdbyWtHSWdIURP"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_key, access_secret) 
api = tweepy.API(auth) 

screenname = ["narendramodi","bogochisaac","SueDHellmann","DrTomFrieden","DrTedros","ScottGottliebMD","T_Inglesby","ashishkjha","COVIDNewsByMIB"]

tweet,tweets_dates,text_query,hyperlinks,screennames_twitter,profile_pic = [], [], ["CORONA","COVID"], [], [], []

count = 0
for i in screenname:
    tweets = api.user_timeline(screen_name=i,count=100,tweet_mode="extended")  
    for status in tweets:
        for text in text_query:
            if text in status.full_text and detect(status.full_text)=="en" and "RT" not in status.full_text:
                tweet.append(status.full_text)
                tweets_dates.append(status.created_at)
                screennames_twitter.append('@'+status.user.screen_name)
                profile_pic.append(status.user.profile_image_url)
                if len(status.entities['urls']) >0:
                    hyperlinks.append(status.entities['urls'][0]['url'])
                else:
                    hyperlinks.append('#')

for i in range(len(tweet)):
    if hyperlinks[i] == '0' and 'https://' in  tweet[i]:
        somet = tweet[i].find("https://")
        hyperlinks[i] = tweet[i][somet::]
        tweet[i] = tweet[i].replace(hyperlinks[i],' ')
        

for i in range(len(tweet)):
    if hyperlinks[i] != '0':
        somet = tweet[i].find("https://")
        hyperlinks[i] = tweet[i][somet::]
        tweet[i] = tweet[i].replace(hyperlinks[i],' ')
        
df = pd.DataFrame({"screenName":screennames_twitter,"Tweets":tweet,"Timeframe":tweets_dates,"Links":hyperlinks,"profilePictue":profile_pic})
df.sort_values(by=['Timeframe'], inplace=True, ascending=False)
df = df.reset_index(drop=True)
df.to_csv('All-Tweets.csv',index=False)   
df

,screenName,Tweets,Timeframe,Links,profilePictue
0,@COVIDNewsByMIB,#IndiaFightsCorona:\n\n📍#Guidelines for prison...,2020-08-17 15:11:03,https://t.co/vpWxKWr90a,http://pbs.twimg.com/profile_images/1244933005...
1,@COVIDNewsByMIB,#IndiaFightsCorona:\n\n📍 Preventing #COVID19 a...,2020-08-17 14:06:48,https://t.co/WNAW6xHkC9,http://pbs.twimg.com/profile_images/1244933005...
2,@COVIDNewsByMIB,#IndiaFightsCorona: \n\n📍Spitting publicly can...,2020-08-17 13:47:31,https://t.co/Pp6MWIknl3,http://pbs.twimg.com/profile_images/1244933005...
3,@COVIDNewsByMIB,#IndiaFightsCorona:\n\n📍Points to remember whi...,2020-08-17 12:13:52,https://t.co/8fbQyU6Uxx,http://pbs.twimg.com/profile_images/1244933005...
4,@DrTedros,@PeterByass supervised my Ph D thesis at the...,2020-08-17 11:59:36,.,http://pbs.twimg.com/profile_images/1242902762...
...,...,...,...,...,...
153,@SueDHellmann,Social distancing will save lives - and will d...,2020-03-26 22:12:41,https://t.co/fZsFOIhXDM,http://pbs.twimg.com/profile_images/1224560043...
154,@SueDHellmann,Mano-a-Mano ! Steph &amp; Dr. Fauci\nNot March...,2020-03-26 01:07:29,https://t.co/CaWWxOUDyp,http://pbs.twimg.com/profile_images/1224560043...
155,@SueDHellmann,Highly recommend piece on where we go next on ...,2020-03-25 19:45:47,https://t.co/hCkcXpHSJC,http://pbs.twimg.com/profile_images/1224560043...
156,@SueDHellmann,Essential to fix this- “Right now we simply ha...,2020-03-24 16:25:05,https://t.co/XmU3XjBcYF,http://pbs.twimg.com/profile_images/1224560043...


In [42]:
newsHeader

[('NZ postpones vote; India coronavirus deaths top 50,000: Live\nReference: Al Jazeera English\nUpdated: 48 minutes ago\n',
  ' For more: https://news.google.com/articles/CAIiENbralxBDPTMmV2V9yL2GukqFAgEKgwIACoFCAowhgIwkDgw0O8B?hl=en-IN&gl=IN&ceid=IN%3Aen'),
 ("India's COVID-19 vaccine: From price to production, companies brief expert panel on progress\nReference: Livemint\nUpdated: 24 minutes ago\n",
  ' For more: https://news.google.com/articles/CAIiEDI0AL4ssaapuaYkXg1znPMqFwgEKg4IACoGCAowxLQ_MNevCDCyrdsG?hl=en-IN&gl=IN&ceid=IN%3Aen'),
 ('Weekly Covid numbers hit new high but growth slows\nReference: Times of India\nUpdated: 10 hours ago\n',
  ' For more: https://news.google.com/articles/CAIiEEfHhxAaQY0zT7m9MBRBgtwqGQgEKhAIACoHCAowzrL9CjDC7vQCMK2y1gU?hl=en-IN&gl=IN&ceid=IN%3Aen'),
 ('When will coronavirus cases peak in India?\nReference: Livemint\nUpdated: 12 hours ago\n',
  ' For more: https://news.google.com/articles/CAIiEAHYrvHtIge3mYSOcVjnOYQqFwgEKg4IACoGCAowxLQ_MNevCDDnvNMF?hl=e

In [2]:
import pandas as pd
df = pd.read_csv('All-Tweets.csv')

In [3]:
df["Tweets"][0]

'#IndiaFightsCorona:\n\n📍#Guidelines for prisons during #COVID19\n\n↗️ Ensure quality health services, including uninterrupted access to the prevention and treatment of HIV, TB, hepatitis etc\n\n↗️ Reduce overcrowding\n\n#StaySafe #IndiaWillWin  '

In [4]:
df[df["Links"]=='0']

,screenName,Tweets,Timeframe,Links,profilePictue


In [5]:
links = []
for i in range(len(df["Tweets"])):
    if df["Links"][i] == '0' and 'https://' in  df["Tweets"][i]:
        somet = df["Tweets"][i].find("https://")
        links.append(df["Tweets"][i][somet::])
        df["Tweets"][i].replace(df["Tweets"][i][somet::],'')
        df['Links'][i] = df["Tweets"][i][somet::]


In [6]:
df[df["Links"]=='0']

,screenName,Tweets,Timeframe,Links,profilePictue


In [7]:
df
df.to_csv('All-Tweets.csv',index=False)

In [8]:
df["profilePictue"][0]

'http://pbs.twimg.com/profile_images/1244933005795135489/bPoYebla_normal.jpg'